**Carga dos Dados Refinados**

In [ ]:
# ==============================================================================
# PROJETO: FlightOnTime - Treinamento de Modelo
# OBJETIVO: Treinar o Random Forest e exportar o arquivo .joblib final.
# AUTORA: Sueli da Hora
# ==============================================================================

from google.colab import auth, drive
from google.cloud import bigquery
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import shutil

auth.authenticate_user()
client = bigquery.Client(project='flightontime-hackathon')

# Carregando a tabela que já passou pela limpeza e engenharia básica
query = "SELECT * FROM `flightontime-hackathon.dados_voos.tabela_treinamento_final` LIMIT 100000"
df = client.query(query).to_dataframe()

print(f"Dados carregados para treinamento: {df.shape[0]} registros.")

Dados carregados para treinamento: 100000 registros.


**Preparação e Treinamento do Modelo**

In [ ]:

from sklearn.preprocessing import LabelEncoder

# 1. Encoding: O modelo não lê 'DL', então transformamos em números novamente
le = LabelEncoder()
colunas_categoricas = ['companhia', 'origem', 'estado_origem', 'destino', 'estado_destino']

for col in colunas_categoricas:
    df[col] = le.fit_transform(df[col].astype(str))

# 2. Preparação das variáveis (Removendo o que não é feature e o alvo)
X = df.drop(columns=['atrasado', 'data_voo'])
y = df['atrasado']

# 3. Divisão estratégica (80% treino / 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Treinamento do Modelo Random Forest
modelo = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
modelo.fit(X_train, y_train)

# 5. Avaliação
y_pred = modelo.predict(X_test)
print(f"Sucesso! Acurácia Validada: {accuracy_score(y_test, y_pred):.2%}")

Sucesso! Acurácia Validada: 82.45%


**Relatório de Avaliação do Modelo**

In [14]:
from sklearn.metrics import classification_report

# Gerando o relatório detalhado de métricas
print("--- Relatório de Avaliação do Modelo ---")
print(classification_report(y_test, y_pred))

--- Relatório de Avaliação do Modelo ---
              precision    recall  f1-score   support

         0.0       0.83      1.00      0.90     16493
         1.0       0.48      0.01      0.02      3507

    accuracy                           0.82     20000
   macro avg       0.65      0.50      0.46     20000
weighted avg       0.76      0.82      0.75     20000



**Persistência de Ativos (Exportação)**

In [15]:
from google.colab import drive
import shutil
import joblib

# 1. Exportar o modelo treinado para um arquivo binário local
joblib.dump(modelo, 'modelo_atraso_voo.joblib')
print("Arquivo .joblib gerado localmente no Colab.")

# 2. Conectar ao Google Drive
drive.mount('/content/drive')

# 3. Copiar o arquivo para o seu Drive pessoal
# Isso garante que o arquivo não suma quando você fechar o Colab
DESTINO_DRIVE = '/content/drive/MyDrive/modelo_atraso_voo.joblib'
shutil.copy('modelo_atraso_voo.joblib', DESTINO_DRIVE)

print(f"Sucesso! Modelo persistido no seu Drive em: {DESTINO_DRIVE}")

Arquivo .joblib gerado localmente no Colab.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sucesso! Modelo persistido no seu Drive em: /content/drive/MyDrive/modelo_atraso_voo.joblib
